In [1]:
from component.CalculoIndiceUnidadeFederativaComponent import CalculoIndiceUnidadeFederativaComponent
from model.constants.ParametrosConstantes import ParametrosConstantes
from service.UnidadeFederativaService import UnidadeFederativaService
from util.BancoDadosUtil import BancoDadosUtil
from util.DataFrameUtil import DataFrameUtil
from util.LoggerUtil import LoggerUtil
from util.WarningUtil import WarningUtil

from pandas import DataFrame
from time import sleep

In [2]:
# Configurações iniciais

WarningUtil.ignorar_warning(categoria=DeprecationWarning)
WarningUtil.ignorar_warning(categoria=UserWarning)

log = LoggerUtil.configurar_logger(arquivo_log=ParametrosConstantes.CAMINHO_LOG_CALCULO_INDICE_UNIDADE_FEDERATIVA)

In [3]:
calculo_indice_component = CalculoIndiceUnidadeFederativaComponent()
unidade_federativa_service = UnidadeFederativaService()

In [4]:
while True:
    log.info(msg="Iniciando o cálculo do indíce de 15 minutos das unidades federativas.")
    conexao_bd = BancoDadosUtil.estabelecer_conexao_banco()

    if calculo_indice_component.verificar_nao_existencia_registros_pendentes(conexao_bd):
        break

    df_municipio = unidade_federativa_service.buscar_registros_pendentes_calculo_indice_15min(conexao_bd)

    log.info(msg="Calculando o índice de 15 minutos para os municípios.")

    df_resultado = DataFrameUtil.processar_dataframe_dask(
        df=df_municipio,
        funcao=calculo_indice_component.processar_particao_dask,
        meta=DataFrame(columns=["codigo_unidade_federativa", "lista_dict_historico_erro", "status"]),
        qtde_particoes=1,
        conexao_bd=conexao_bd
    )

    calculo_indice_component.persistir_resultado(df_resultado, conexao_bd)

    BancoDadosUtil.encerrar_conexao_banco(conexao_bd)
    sleep(10)

2024-07-08 12:04:14,165 - INFO - 2218751824 - Iniciando o cálculo do indíce de 15 minutos das unidades federativas.
2024-07-08 12:04:14,428 - INFO - BancoDadosUtil - A conexão com o banco de dados BDTCCPRD foi estabelecida com sucesso.
2024-07-08 12:04:14,428 - INFO - CalculoIndiceUnidadeFederativaComponent - Há um total de 27 unidades federativas a serem processadas.
2024-07-08 12:04:14,428 - INFO - 2218751824 - Calculando o índice de 15 minutos para os municípios.
2024-07-08 12:04:15,287 - INFO - CalculoIndiceUnidadeFederativaComponent - Calculando índice de 15 minutos para a unidade federativa 11 - Rondônia.
2024-07-08 12:04:15,308 - ERROR - CalculoIndiceService - Houve um erro ao calcular o índice de 15 minutos para a unidade_federativa 11. Erro: IntegrityError na linha 30 do arquivo c:\Users\thiag\Documents\GitHub\TCC-cidade-de-15-minutos\src\main\service\CalculoIndiceService.py: (psycopg2.errors.UniqueViolation) ERRO:  duplicar valor da chave viola a restrição de unicidade "t_ind

In [5]:
BancoDadosUtil.encerrar_conexao_banco(conexao_bd)

2024-07-08 12:04:25,679 - INFO - BancoDadosUtil - Encerrando a conexão com o banco de dados.
2024-07-08 12:04:25,682 - INFO - BancoDadosUtil - A conexão com o banco de dados foi encerrada com sucesso.
